# ハルカ 小遙

## sp計算方式 - 二代進化 L.v10~24

所有參數

In [67]:
import sqlite3
import pandas as pd
import datetime

class SamePokemon():
    def __init__(self):
        self.conn = sqlite3.connect("../pokemon_database.db")
        self.cursor = self.conn.cursor()
        self.cursor.execute('SELECT * FROM pokemon')
        self.data = self.cursor.fetchall()

    def get_data_as_dataframe(self):
        columns = [
            "id",
            "img_num",
            "name",
            "sp",
            "expertise",
            "level",
            "help_fruit",
            "help_fruit_num",
            "help_ingredient_1",
            "help_ingredient_num_1",
            "help_ingredient_2",
            "help_ingredient_num_2",
            "help_ingredient_3",
            "help_ingredient_num_3",
            "help_time",
            "help_max",
            "skill_main",
            "skill_main_num",
            "skill_main_level",
            "skill_second_1",
            "skill_second_num_1",
            "skill_second_2",
            "skill_second_num_2",
            "skill_second_3",
            "skill_second_num_3",
            "skill_second_4",
            "skill_second_num_4",
            "skill_second_5",
            "skill_second_num_5",
            "power_up",
            "power_down",
            "備註"
        ]

        df = pd.DataFrame(self.data, columns=columns)
        return df

# 創建一個物件
pokemon_instance = SamePokemon()

# 將資料以 DataFrame 格式讀取
pokemon_data = pokemon_instance.get_data_as_dataframe()

# 取得時間字串
time_strings = pokemon_data['help_time']

def convert_to_time_format(time_str):
    # 將時間字串轉換為時間物件
    time_obj = datetime.datetime.strptime(time_str, '%H:%M:%S').time()
    # 將時間物件轉換為整數，以時 * 10000 + 分 * 100 + 秒格式表示
    return time_obj.hour * 10000 + time_obj.minute * 100 + time_obj.second

# 將時間字串轉換為整數格式
time_converted = [convert_to_time_format(time_str) for time_str in time_strings]

# 更新 'help_time' 欄位
time_indices =list(range(7, 8))+list(range(9, 11))+ list(range(19, 21))+list(range(22, 23))+list(range(31, 32))+ list(range(44, 46))+list(range(47, 48))+ list(range(91, 99))

for i, idx in enumerate(time_indices):
    pokemon_data.at[idx, 'help_time'] = time_converted[i]


In [68]:
# 取y值
y = pokemon_data.iloc[list(range(7, 8))+list(range(9, 11))+ list(range(19, 21))+list(range(22, 23))+list(range(31, 32))+ list(range(44, 46))+list(range(47, 48))+ list(range(91, 99)), 3]

In [69]:
#取X值
X = pokemon_data.iloc[list(range(7, 8))+list(range(9, 11))+ list(range(19, 21))+list(range(22, 23))+list(range(31, 32))+ list(range(44, 46))+list(range(47, 48))+ list(range(91, 99)), [5, 7, 9] + list(range(14, 21))+ list(range(29,31))]

In [70]:
# 對 skill_main 進行 Label Encoding
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
X['skill_main_encoded'] = label_encoder.fit_transform(X['skill_main'])
# 除去原始的 skill_main 欄位
X = X.drop('skill_main', axis=1)

label_encoder = LabelEncoder()
X['skill_second_1_encoded'] = label_encoder.fit_transform(X['skill_second_1'])
# 除去原始的 skill_second_1 欄位
X = X.drop('skill_second_1', axis=1)

# 繼續對其他類別型欄位做 One-Hot Encoding
categorical_columns = ['power_up', 'power_down']

for col in categorical_columns:
    X = pd.concat([X, pd.get_dummies(X[col], prefix=col, drop_first=True)], axis=1)
    X = X.drop(col, axis=1)

## 把數據分割成訓練集&測試集

In [78]:
from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.4,random_state=2)
# test_size=0.3 = 隨意抓取30%測試集
# random_state 重跑訓練值幾遍

In [79]:
from sklearn.linear_model import LinearRegression 
regressor=LinearRegression() # 初始化
model=regressor.fit(X_train,y_train)

In [80]:
y_predict=regressor.predict(X_test)

from sklearn.metrics import r2_score # 2的平方越大(越接近1)，多元線性回歸就越好
score1=r2_score(y_test,y_predict) # 比較訓練集&測試集之間的預測好不好

In [81]:
score1

0.6539110301750009

In [82]:
model.coef_ # y=b+7.03471165e*data1+1.20332163e*data2+3.96425367e*data3+1.53936312e*data4 -4.80880533e*data5

array([ 9.64118456e+00, -6.66900709e+01, -1.46905338e+01,  9.41230433e-03,
        8.17312018e+00,  2.62544560e-03,  5.51887650e+01, -2.60057067e-01,
       -5.67737967e+01,  2.82747046e+01,  1.31768217e+01, -3.78695619e+01,
       -1.42440789e+01,  6.15015679e+00,  0.00000000e+00, -1.19477609e+01,
       -1.42440789e+01,  4.22069842e+01])

In [83]:
model.intercept_ #b=74.48195300240218

476.6320011717829

In [84]:
model.score(X,y) # 再次評估loss值 =>跟score1比較=>數據可用

0.829333152312992